In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade pip
!pip install --upgrade h5py
!pip install --upgrade tensorflow
!pip install imblearn
!pip install keras-tuner


In [ ]:

"""
Stabilized Gaze Analysis System - Main Script
"""

import os
import sys
import json
import shutil
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import logging
from pathlib import Path
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import time

try:
    from google.colab import drive
except ImportError:
    drive = None

class LogCapture:
    def __init__(self, log_file_path: Path) -> None:
        self.log_file_path = log_file_path
        self.logger = None
        self.console_handler = None
        self.file_handler = None

    def setup_logging(self) -> None:
        self.logger = logging.getLogger('GazeAnalysisSystem')
        self.logger.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        self.file_handler = logging.FileHandler(str(self.log_file_path))
        self.file_handler.setLevel(logging.INFO)
        self.file_handler.setFormatter(formatter)
        self.console_handler = logging.StreamHandler(sys.stdout)
        self.console_handler.setLevel(logging.INFO)
        self.console_handler.setFormatter(formatter)
        self.logger.addHandler(self.file_handler)
        self.logger.addHandler(self.console_handler)

    def cleanup(self) -> None:
        if self.logger:
            self.logger.removeHandler(self.file_handler)
            self.logger.removeHandler(self.console_handler)
            self.file_handler.close()
            self.console_handler.close()

class GradientDebugCallback(tf.keras.callbacks.Callback):
    """Custom callback to monitor gradient values during training."""
    def __init__(self, logger):
        super().__init__()
        self.logger = logger

    def on_batch_end(self, batch, logs=None):
        try:
            # Get trainable weights
            weights = self.model.trainable_weights

            # Create gradient tape context
            with tf.GradientTape() as tape:
                # Get a batch from the training data
                for x, y in logs.get('dataset', []):
                    # ADD THIS DATA INSPECTION BLOCK
                    self.logger.info(f"Batch {batch}: Input data (x) stats:")
                    self.logger.info(f"  Min: {tf.reduce_min(x).numpy()}")
                    self.logger.info(f"  Max: {tf.reduce_max(x).numpy()}")
                    self.logger.info(f"  Mean: {tf.reduce_mean(x).numpy()}")
                    self.logger.info(f"  Std: {tf.math.reduce_std(x).numpy()}")
                    self.logger.info(f"Batch {batch}: Target data (y) stats:")
                    self.logger.info(f"  Min: {tf.reduce_min(y).numpy()}")
                    self.logger.info(f"  Max: {tf.reduce_max(y).numpy()}")
                    self.logger.info(f"  Mean: {tf.reduce_mean(y).numpy()}")
                    self.logger.info(f"  Std: {tf.math.reduce_std(y).numpy()}")
                    if tf.reduce_any(tf.math.is_nan(x)):
                        self.logger.warning(f"Batch {batch}: NaN in input data (x)")
                    if tf.reduce_any(tf.math.is_inf(x)):
                        self.logger.warning(f"Batch {batch}: Inf in input data (x)")
                    if tf.reduce_any(tf.math.is_nan(y)):
                        self.logger.warning(f"Batch {batch}: NaN in target data (y)")
                    if tf.reduce_any(tf.math.is_inf(y)):
                        self.logger.warning(f"Batch {batch}: Inf in target data (y)")

                    predictions = self.model(x, training=True)
                    loss = self.model.compiled_loss(y, predictions)
                    break  # Just use first batch

            # Calculate gradients
            if hasattr(tape, 'gradient'):
                grads = tape.gradient(loss, weights)

                # Check for NaN/Inf values
                has_nan = any(tf.reduce_any(tf.math.is_nan(g)) for g in grads if g is not None)
                has_inf = any(tf.reduce_any(tf.math.is_inf(g)) for g in grads if g is not None)

                if has_nan or has_inf:
                    self.logger.warning(f"Batch {batch}: NaN/Inf gradients detected")
                    for i, (w, g) in enumerate(zip(weights, grads)):
                        if g is not None and (tf.reduce_any(tf.math.is_nan(g)) or tf.reduce_any(tf.math.is_inf(g))):
                            self.logger.warning(f"Layer {i}: {w.name}")

        except Exception as e:
            # Log but don't stop training
            self.logger.warning(f"Error in gradient monitoring: {str(e)}")
            pass

class NanLossCallback(tf.keras.callbacks.Callback):
    """Callback to monitor for NaN losses."""
    def __init__(self, logger):
        super().__init__()
        self.logger = logger

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        if 'loss' in logs and (np.isnan(logs['loss']) or np.isinf(logs['loss'])):
            self.logger.warning(f"NaN/Inf loss detected at batch {batch}: {logs['loss']}")
            self.model.stop_training = True

class GazeAnalysisSystem:
    def __init__(
        self,
        gaze_data_path: str = '/content/drive/MyDrive/pilotdata',
        scores_data_path: str = '/content/drive/MyDrive/output',
        window_size: int = 64,
        stride: int = 1,
        model_type: str = "cnn_transformer",
        batch_size: int = 64,
        use_robust_scaler: bool = True,
        clip_range: tuple = (-3, 3),
        max_seq_per_participant: int = 50000
    ) -> None:
        # Create a run folder with timestamp
        run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.run_folder = Path(scores_data_path) / f'run_{run_timestamp}'
        self.run_folder.mkdir(parents=True, exist_ok=True)

        # Setup logging
        log_dir = self.run_folder / 'logs'
        log_dir.mkdir(exist_ok=True)
        self.log_file_path = log_dir / f'gaze_analysis_{run_timestamp}.log'
        self.log_capture = LogCapture(self.log_file_path)
        self.log_capture.setup_logging()
        self.logger = logging.getLogger('GazeAnalysisSystem')
        self.logger.info("Initializing GazeAnalysisSystem")

        # Mount drive if in Colab
        if drive and not Path('/content/drive/MyDrive').exists():
            drive.mount('/content/drive')

        # Store configuration
        self.gaze_data_path = Path(gaze_data_path)
        self.scores_data_path = Path(scores_data_path)
        self.sequence_length = window_size
        self.stride = stride
        self.batch_size = batch_size
        self.model_type = model_type.lower()
        self.use_robust_scaler = use_robust_scaler
        self.clip_range = clip_range
        self.max_seq_per_participant = max_seq_per_participant

        # Define features and configuration
        self.features = [
            'FPOGX', 'FPOGY', 'FPOGD',
            'BPOGX', 'BPOGY',
            'LPCX', 'LPCY', 'LPD', 'LPS',
            'RPCX', 'RPCY', 'RPD', 'RPS',
            'LPMM', 'RPMM',
            'CX', 'CY'
        ]
        self.selected_features = self.features
        self.target = 'Row_Total_Score'
        self.timestamp_column = 'TIMESTAMP'

        # Initialize scalers
        self.scaler = None
        self.target_scaler = MinMaxScaler()

        self.logger.info("Initialization complete")

    def __del__(self):
        if hasattr(self, 'log_capture'):
            self.log_capture.cleanup()

    def get_participant_ids(self) -> list:
        """Extract participant IDs from gaze data filenames."""
        try:
            self.logger.info("Getting participant IDs")
            all_files = os.listdir(self.gaze_data_path)
            gaze_files = [f for f in all_files if f.endswith('_all_gaze.csv')]
            if not gaze_files:
                raise ValueError(f"No '_all_gaze.csv' files found in {self.gaze_data_path}")
            participant_ids = [f.split('_')[0] for f in gaze_files]
            self.logger.info(f"Found {len(participant_ids)} participants: {participant_ids}")
            return participant_ids
        except Exception as e:
            self.logger.exception("Error getting participant IDs")
            raise

    def validate_and_clean_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """Validate and clean data by handling outliers and invalid values."""
        try:
            self.logger.info("Starting data validation and cleaning")

            # Check for infinite or NaN values
            for feature in self.features:
                inf_mask = np.isinf(data[feature])
                nan_mask = np.isnan(data[feature])
                if inf_mask.any():
                    self.logger.warning(f"Infinite values found in {feature}: {inf_mask.sum()} values")
                    data.loc[inf_mask, feature] = data[feature].median()
                if nan_mask.any():
                    self.logger.warning(f"NaN values found in {feature}: {nan_mask.sum()} values")
                    data.loc[nan_mask, feature] = data[feature].median()

            # Handle outliers using IQR method
            for feature in self.features:
                q1 = data[feature].quantile(0.25)
                q3 = data[feature].quantile(0.75)
                iqr = q3 - q1
                lower_bound = q1 - 3 * iqr
                upper_bound = q3 + 3 * iqr
                outliers = (data[feature] < lower_bound) | (data[feature] > upper_bound)
                if outliers.any():
                    self.logger.warning(f"Outliers found in {feature}: {outliers.sum()} values")
                    data.loc[outliers, feature] = data[feature].median()

            # Log statistics after cleaning
            for feature in self.features:
                stats = data[feature].describe()
                self.logger.info(f"Statistics for {feature}:")
                self.logger.info(f"  Mean: {stats['mean']:.3f}")
                self.logger.info(f"  Std: {stats['std']:.3f}")
                self.logger.info(f"  Min: {stats['min']:.3f}")
                self.logger.info(f"  Max: {stats['max']:.3f}")

            return data

        except Exception as e:
            self.logger.exception("Error in validate_and_clean_data")
            raise

    def align_data_at_intercept(self, df_gaze: pd.DataFrame, df_scores: pd.DataFrame):
        """Align gaze and score data using an intercept time based on score phases."""
        intercept_time = 0
        if 'Phase' in df_scores.columns and 'Time' in df_scores.columns:
            final_rows = df_scores[df_scores['Phase'] == 'Final']
            if not final_rows.empty:
                intercept_time = final_rows.iloc[0]['Time']
            else:
                stepdown = df_scores[df_scores['Phase'] == 'Stepdown']
                if not stepdown.empty:
                    intercept_time = stepdown.iloc[0]['Time']

        df_scores = df_scores.copy()
        df_scores['RelTime'] = df_scores['Time'] - intercept_time
        df_scores = df_scores[df_scores['RelTime'] >= 0].reset_index(drop=True)

        df_gaze = df_gaze.copy()
        df_gaze['RelTime'] = df_gaze[self.timestamp_column] - intercept_time
        df_gaze = df_gaze[df_gaze['RelTime'] >= 0].reset_index(drop=True)

        df_scores.sort_values('RelTime', inplace=True)
        df_gaze.sort_values('RelTime', inplace=True)
        return df_gaze, df_scores, intercept_time

    def load_and_preprocess_data(self) -> pd.DataFrame:
        """Load and preprocess gaze and score data for all participants with improved stability."""
        try:
            participant_ids = self.get_participant_ids()
            all_data = []
            processed = 0

            for pid in participant_ids:
                self.logger.info(f"Processing participant: {pid}")
                gaze_file = self.gaze_data_path / f"{pid}_all_gaze.csv"
                if not gaze_file.exists():
                    self.logger.warning(f"No gaze file for {pid}, skipping...")
                    continue

                # Load and validate gaze data
                try:
                    df_gaze = pd.read_csv(gaze_file)
                    self.logger.info(f"Loaded gaze data for {pid}: {df_gaze.shape}")
                except Exception as e:
                    self.logger.error(f"Error reading gaze file for {pid}: {str(e)}")
                    continue

                # Handle time column
                time_cols = [c for c in df_gaze.columns if c.startswith("TIME(")]
                if not time_cols:
                    self.logger.warning(f"No TIME(...) column for {pid}, skipping...")
                    continue
                time_col = time_cols[0]
                df_gaze.rename(columns={time_col: 'TimeRaw'}, inplace=True)

                # Create relative timestamp and validate
                try:
                    df_gaze['TIMESTAMP'] = df_gaze['TimeRaw'] - df_gaze['TimeRaw'].iloc[0]
                    if df_gaze['TIMESTAMP'].isna().any():
                        self.logger.warning(f"NaN timestamps found for {pid}, fixing...")
                        df_gaze['TIMESTAMP'] = df_gaze['TIMESTAMP'].interpolate(method='linear')
                except Exception as e:
                    self.logger.error(f"Error processing timestamps for {pid}: {str(e)}")
                    continue

                # Load and validate scores data
                scores_file = self.scores_data_path / pid / f"{pid}_detailed_scores.csv"
                if not scores_file.exists():
                    self.logger.warning(f"No detailed scores for {pid}, skipping...")
                    continue

                try:
                    df_scores = pd.read_csv(scores_file)
                    self.logger.info(f"Loaded scores data for {pid}: {df_scores.shape}")
                except Exception as e:
                    self.logger.error(f"Error reading scores file for {pid}: {str(e)}")
                    continue

                # Process scores timestamps
                try:
                    df_scores['Time'] = df_scores['Time'] - df_scores['Time'].iloc[0]
                    df_scores['TIMESTAMP'] = df_scores['Time']
                except Exception as e:
                    self.logger.error(f"Error processing score timestamps for {pid}: {str(e)}")
                    continue

                # Align data and validate
                try:
                    df_gaze_align, df_scores_align, intercept = self.align_data_at_intercept(df_gaze, df_scores)
                    self.logger.info(f"Aligned data at intercept time: {intercept}")
                except Exception as e:
                    self.logger.error(f"Error aligning data for {pid}: {str(e)}")
                    continue

                # Prepare aligned data
                df_gaze_align.drop(columns=['TIMESTAMP'], inplace=True, errors='ignore')
                df_scores_align.drop(columns=['TIMESTAMP'], inplace=True, errors='ignore')
                df_gaze_align.rename(columns={'RelTime': 'TIMESTAMP'}, inplace=True)
                df_scores_align.rename(columns={'RelTime': 'TIMESTAMP'}, inplace=True)
                df_gaze_align.sort_values('TIMESTAMP', inplace=True)
                df_scores_align.sort_values('TIMESTAMP', inplace=True)

                # Merge data with validation
                try:
                    df_merged = pd.merge_asof(
                        df_gaze_align,
                        df_scores_align[['TIMESTAMP', 'Row_Total_Score']],
                        on='TIMESTAMP',
                        direction='nearest',
                        tolerance=1.0
                    )

                    # Validate merge results
                    if df_merged.empty:
                        self.logger.warning(f"Empty merged data for {pid}, skipping...")
                        continue

                    missing_score = df_merged['Row_Total_Score'].isna().sum()
                    if missing_score > 0:
                        self.logger.warning(f"Missing scores in merged data for {pid}: {missing_score} rows")
                        df_merged.dropna(subset=['Row_Total_Score'], inplace=True)

                    # Handle missing feature values
                    for feature in self.features:
                        missing_count = df_merged[feature].isna().sum()
                        if missing_count > 0:
                            self.logger.warning(f"Missing values in {feature} for {pid}: {missing_count} rows")
                            df_merged[feature] = df_merged[feature].interpolate(method='linear')

                    # Final validation
                    if df_merged[self.features].isna().any().any():
                        self.logger.warning(f"Remaining NaN values for {pid}, using ffill/bfill")
                        df_merged[self.features] = df_merged[self.features].ffill().bfill()

                    df_merged['participant_id'] = pid

                    # Log statistics for this participant
                    self.logger.info(f"Processed data for {pid}:")
                    self.logger.info(f"  Final shape: {df_merged.shape}")
                    self.logger.info(f"  Score range: {df_merged['Row_Total_Score'].min():.2f} to {df_merged['Row_Total_Score'].max():.2f}")

                    all_data.append(df_merged)
                    processed += 1

                except Exception as e:
                    self.logger.error(f"Error merging data for {pid}: {str(e)}")
                    continue

            if processed == 0:
                raise ValueError("No participants processed successfully")

            # Combine all data
            data = pd.concat(all_data, ignore_index=True)
            self.logger.info(f"Combined data shape: {data.shape}")

            # Validate and clean combined data
            data = self.validate_and_clean_data(data)

            # Scale features
            if self.use_robust_scaler:
                self.scaler = RobustScaler()
                self.logger.info("Using RobustScaler for feature scaling")


                for feature in self.features:
                    q1 = data[feature].quantile(0.25)
                    q3 = data[feature].quantile(0.75)
                    iqr = q3 - q1
                    if iqr == 0:
                        self.logger.warning(f"Feature '{feature}' has zero IQR.  This might cause issues with RobustScaler.")


            else:
                self.scaler = StandardScaler()
                self.logger.info("Using StandardScaler for feature scaling")


            # Scale features with validation
            try:
                scaled_features = self.scaler.fit_transform(data[self.features])
                clip_low, clip_high = self.clip_range
                scaled_features = np.clip(scaled_features, clip_low, clip_high)

                for i, feature in enumerate(self.features):
                    data[feature] = scaled_features[:, i]

                # Verify scaling results
                for feature in self.features:
                    stats = data[feature].describe()
                    self.logger.info(f"Scaled {feature}:")
                    self.logger.info(f"  Mean: {stats['mean']:.3f}")
                    self.logger.info(f"  Std: {stats['std']:.3f}")
                    self.logger.info(f"  Range: [{stats['min']:.3f}, {stats['max']:.3f}]")
                    # Check for any remaining NaN or inf values.
                    if data[feature].isnull().any():
                        self.logger.error(f"NaN values found in {feature} after scaling!")
                    if np.isinf(data[feature]).any():
                        self.logger.error(f"Inf values found in {feature} after scaling!")

            except Exception as e:
                self.logger.error(f"Error scaling features: {str(e)}")
                raise

            # Scale target variable
            try:
                target_values = data[self.target].values.reshape(-1, 1)
                data[self.target] = self.target_scaler.fit_transform(target_values)

                # Verify target scaling
                stats = data[self.target].describe()
                self.logger.info(f"Scaled target variable:")
                self.logger.info(f"  Mean: {stats['mean']:.3f}")
                self.logger.info(f"  Std: {stats['std']:.3f}")
                self.logger.info(f"  Range: [{stats['min']:.3f}, {stats['max']:.3f}]")

            except Exception as e:
                self.logger.error(f"Error scaling target variable: {str(e)}")
                raise

            # Save scalers
            try:
                scaler_path = self.run_folder / 'feature_scaler.gz'
                target_scaler_path = self.run_folder / 'target_scaler.gz'
                joblib.dump(self.scaler, scaler_path)
                joblib.dump(self.target_scaler, target_scaler_path)
                self.logger.info(f"Saved scalers to {self.run_folder}")
            except Exception as e:
                self.logger.error(f"Error saving scalers: {str(e)}")
                raise

            self.data = data
            return data

        except Exception as e:
            self.logger.exception("Critical error in load_and_preprocess_data")
            raise

    def build_model(self, sequence_length: int, n_features: int) -> tf.keras.Model:
        """Build model with balanced regularization."""
        try:
            # Define a moderate regularizer
            regularizer = tf.keras.regularizers.L2(l2=1e-5)  # Reduced from 1e-4

            # Define optimizer
            optimizer = tf.keras.optimizers.Adam(
                learning_rate=1e-4,
                clipnorm=0.5
            )

            if self.model_type == "cnn":
                inputs = tf.keras.layers.Input(shape=(sequence_length, n_features))


                x = tf.keras.layers.BatchNormalization()(inputs)

                # First conv block
                conv1 = tf.keras.layers.Conv1D(
                    32, kernel_size=3, padding="same",
                    kernel_regularizer=regularizer,
                    kernel_initializer='he_normal'
                )(x)
                conv1 = tf.keras.layers.BatchNormalization()(conv1)
                conv1 = tf.keras.layers.Activation("relu")(conv1)
                conv1 = tf.keras.layers.Dropout(0.1)(conv1)

                # Second conv block
                conv2 = tf.keras.layers.Conv1D(
                    64, kernel_size=3, padding="same",
                    kernel_regularizer=regularizer,
                    kernel_initializer='he_normal'
                )(conv1)
                conv2 = tf.keras.layers.BatchNormalization()(conv2)
                conv2 = tf.keras.layers.Activation("relu")(conv2)
                conv2 = tf.keras.layers.Dropout(0.1)(conv2)

                # Global pooling and final dense layers
                x = tf.keras.layers.GlobalAveragePooling1D()(conv2)
                x = tf.keras.layers.Dense(32, activation="relu", kernel_regularizer=regularizer)(x)
                x = tf.keras.layers.BatchNormalization()(x)
                outputs = tf.keras.layers.Dense(1, activation="linear")(x)

            elif self.model_type == "lstm":
                inputs = tf.keras.layers.Input(shape=(sequence_length, n_features))
                x = tf.keras.layers.BatchNormalization()(inputs)

                x = tf.keras.layers.LSTM(
                    32, return_sequences=True,
                    kernel_regularizer=regularizer,
                    recurrent_regularizer=regularizer,
                    recurrent_dropout=0.0
                )(x)
                x = tf.keras.layers.BatchNormalization()(x)
                x = tf.keras.layers.Dropout(0.1)(x)

                x = tf.keras.layers.LSTM(
                    16,
                    kernel_regularizer=regularizer,
                    recurrent_regularizer=regularizer,
                    recurrent_dropout=0.0
                )(x)
                x = tf.keras.layers.BatchNormalization()(x)
                x = tf.keras.layers.Dense(16, activation="relu", kernel_regularizer=regularizer)(x)
                x = tf.keras.layers.BatchNormalization()(x)
                outputs = tf.keras.layers.Dense(1, activation="linear")(x)

            elif self.model_type == "cnn_transformer":
                inputs = tf.keras.layers.Input(shape=(sequence_length, n_features))
                x = tf.keras.layers.BatchNormalization()(inputs)

                # Initial projection
                x = tf.keras.layers.Dense(
                    32,
                    kernel_initializer=tf.keras.initializers.HeNormal(),
                    bias_initializer='zeros',
                    kernel_regularizer=regularizer
                )(x)
                x = tf.keras.layers.BatchNormalization()(x)

                # Convolutional feature extraction
                for filters in [32, 64]:
                    x = tf.keras.layers.Conv1D(
                        filters=filters,
                        kernel_size=3,
                        padding="same",
                        kernel_initializer=tf.keras.initializers.HeNormal(),
                        bias_initializer='zeros',
                        kernel_regularizer=regularizer
                    )(x)
                    x = tf.keras.layers.BatchNormalization()(x)
                    x = tf.keras.layers.Activation('relu')(x)
                    x = tf.keras.layers.Dropout(0.1)(x)

                # Transformer block with scaled attention
                attention_output = tf.keras.layers.MultiHeadAttention(
                    key_dim=32,
                    num_heads=4,
                    dropout=0.1,
                    kernel_initializer=tf.keras.initializers.HeNormal()
                )(x, x)
                x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attention_output * 0.5)

                # Feed-forward network with residual connection
                ffn = tf.keras.layers.Dense(
                    64,
                    activation='relu',
                    kernel_initializer=tf.keras.initializers.HeNormal(),
                    bias_initializer='zeros',
                    kernel_regularizer=regularizer
                )(x)
                ffn = tf.keras.layers.BatchNormalization()(ffn)
                ffn = tf.keras.layers.Dropout(0.1)(ffn)
                x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + ffn * 0.5)

                # Output layers
                x = tf.keras.layers.GlobalAveragePooling1D()(x)
                x = tf.keras.layers.BatchNormalization()(x)
                x = tf.keras.layers.Dense(
                    32,
                    activation='relu',
                    kernel_initializer=tf.keras.initializers.HeNormal(),
                    bias_initializer='zeros',
                    kernel_regularizer=regularizer
                )(x)
                x = tf.keras.layers.BatchNormalization()(x)
                x = tf.keras.layers.Dropout(0.1)(x)
                outputs = tf.keras.layers.Dense(
                    1,
                    activation='linear',
                    kernel_initializer=tf.keras.initializers.HeNormal(),
                    bias_initializer='zeros'
                )(x)

            else:
                self.logger.error(f"Unknown model_type '{self.model_type}'. Defaulting to CNN.")
                return self.build_model(sequence_length, n_features)

            model = tf.keras.Model(inputs=inputs, outputs=outputs)

            loss = tf.keras.losses.MeanSquaredError()

            # Gradient monitoring callback
            self.additional_callbacks = [
                GradientDebugCallback(self.logger),
                NanLossCallback(self.logger)
            ]

            # Compile with stable configuration
            model.compile(
                optimizer=optimizer,
                loss=loss,
                metrics=[
                    tf.keras.metrics.MeanAbsoluteError(),
                    tf.keras.metrics.MeanSquaredError()
                ]
            )

            self.logger.info(f"Built stabilized {self.model_type.upper()} model with input shape ({sequence_length}, {n_features})")
            return model

        except Exception as e:
            self.logger.exception("Error building model")
            return None


    def create_training_config(self, train_dataset):
        """Create a stable training configuration with improved monitoring."""
        callbacks = [
            # Early stopping with restored weights
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),

            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.2,
                patience=3,
                verbose=1,
                min_lr=1e-7
            ),

            # Model checkpoint
            tf.keras.callbacks.ModelCheckpoint(
                filepath=str(self.run_folder / 'checkpoints' / 'model_{epoch:02d}.weights.h5'),
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True,
                verbose=1
            ),

            # Simple loss monitoring
            NanLossCallback(self.logger),
            tf.keras.callbacks.TensorBoard(log_dir=str(self.run_folder / 'tensorboard_logs'), histogram_freq=1)
        ]

        return callbacks
    def evaluate_participant_generalization(self):
        """Evaluate how well the model generalizes to entirely new participants."""
        try:
            participant_ids = self.data['participant_id'].unique()
            n_participants = len(participant_ids)

            self.logger.info(f"Performing leave-one-participant-out evaluation with {n_participants} participants")

            # Prepare results tracking
            participant_metrics = []

            for test_pid in participant_ids:
                self.logger.info(f"Using participant {test_pid} as test set")

                # Split data
                train_mask = self.data['participant_id'] != test_pid
                test_mask = self.data['participant_id'] == test_pid

                # Create sequences for training data
                train_data = self.data[train_mask]
                X_train, y_train = self._create_sequences(train_data)

                # Create sequences for test participant
                test_data = self.data[test_mask]
                X_test, y_test = self._create_sequences(test_data)

                # Scale data using only training data
                if self.use_robust_scaler:
                    scaler = RobustScaler().fit(X_train.reshape(-1, X_train.shape[-1]))
                else:
                    scaler = StandardScaler().fit(X_train.reshape(-1, X_train.shape[-1]))

                target_scaler = MinMaxScaler().fit(y_train.reshape(-1, 1))

                # Transform data
                X_train_scaled = scaler.transform(X_train.reshape(-1, X_train.shape[-1]))
                X_train_scaled = np.clip(X_train_scaled, self.clip_range[0], self.clip_range[1])
                X_train_scaled = X_train_scaled.reshape(X_train.shape)

                X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1]))
                X_test_scaled = np.clip(X_test_scaled, self.clip_range[0], self.clip_range[1])
                X_test_scaled = X_test_scaled.reshape(X_test.shape)

                y_train_scaled = target_scaler.transform(y_train.reshape(-1, 1)).ravel()
                y_test_scaled = target_scaler.transform(y_test.reshape(-1, 1)).ravel()

                # Build and train model
                model = self.build_model(X_train.shape[1], X_train.shape[2])

                # Create datasets
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_scaled))
                train_dataset = train_dataset.shuffle(10000).batch(self.batch_size).prefetch(tf.data.AUTOTUNE)

                test_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_scaled))
                test_dataset = test_dataset.batch(self.batch_size).prefetch(tf.data.AUTOTUNE)

                # Train
                model.fit(
                    train_dataset,
                    epochs=50,
                    callbacks=self.create_training_config(train_dataset),
                    verbose=1
                )

                # Evaluate
                test_results = model.evaluate(test_dataset, verbose=1)

                # Record metrics
                participant_metrics.append({
                    'participant_id': test_pid,
                    'test_loss': float(test_results[0]),
                    'test_mae': float(test_results[1]),
                    'test_mse': float(test_results[2])
                })

                # Clean up
                tf.keras.backend.clear_session()

            # Save and analyze results
            metrics_df = pd.DataFrame(participant_metrics)
            metrics_df.to_csv(self.run_folder / 'participant_generalization.csv', index=False)

            # Log summary statistics
            self.logger.info("Participant generalization results:")
            self.logger.info(f"Average MAE: {metrics_df['test_mae'].mean():.4f}")
            self.logger.info(f"Std Dev MAE: {metrics_df['test_mae'].std():.4f}")
            self.logger.info(f"Min MAE: {metrics_df['test_mae'].min():.4f} (Participant {metrics_df.iloc[metrics_df['test_mae'].argmin()]['participant_id']})")
            self.logger.info(f"Max MAE: {metrics_df['test_mae'].max():.4f} (Participant {metrics_df.iloc[metrics_df['test_mae'].argmax()]['participant_id']})")

            return metrics_df

        except Exception as e:
            self.logger.exception("Error in participant generalization evaluation")
            raise
    def _create_sequences(self, df):
        """Helper method to create sequences from a dataframe."""
        feature_data = df[self.features].values
        n_samples = len(df)

        if n_samples < self.sequence_length:
            return np.array([]), np.array([])

        n_sequences = (n_samples - self.sequence_length + self.stride) // self.stride
        sequences = np.zeros((n_sequences, self.sequence_length, len(self.features)))
        targets = np.zeros(n_sequences)

        for i in range(n_sequences):
            start_idx = i * self.stride
            end_idx = start_idx + self.sequence_length
            sequences[i] = feature_data[start_idx:end_idx]
            targets[i] = df[self.target].values[end_idx - 1]

        return sequences, targets
    def train_and_evaluate(self) -> tf.keras.Model:
        """Train the model using cross-validation with GPU optimization."""
        try:
            best_model = None
            best_val_metric = float('inf')
            best_fold = None
            fold_metrics = []
            histories = []

            # Create sequences (keeping existing sequence creation code)
            self.logger.info("Starting sequence creation...")
            participant_ids = self.data['participant_id'].unique()
            X_list, y_list, timestamps_list, participant_ids_list = [], [], [], []

            for pid in participant_ids:
                try:
                    self.logger.info(f"Processing sequences for participant {pid}")
                    df_participant = self.data[self.data['participant_id'] == pid]

                    # Calculate sequences
                    feature_data = df_participant[self.features].values
                    n_samples = len(df_participant)

                    if n_samples < self.sequence_length:
                        self.logger.warning(f"Insufficient samples for participant {pid}")
                        continue

                    n_sequences = (n_samples - self.sequence_length + self.stride) // self.stride
                    sequences = np.zeros((n_sequences, self.sequence_length, len(self.features)))
                    targets = np.zeros(n_sequences)
                    timestamps = np.zeros(n_sequences)

                    for i in range(n_sequences):
                        start_idx = i * self.stride
                        end_idx = start_idx + self.sequence_length
                        sequences[i] = feature_data[start_idx:end_idx]
                        targets[i] = df_participant[self.target].values[end_idx - 1]
                        timestamps[i] = df_participant[self.timestamp_column].values[end_idx - 1]


                    max_sequences = min(self.max_seq_per_participant, 40000)
                    if n_sequences > max_sequences:
                        indices = np.random.choice(n_sequences, max_sequences, replace=False)
                        sequences = sequences[indices]
                        targets = targets[indices]
                        timestamps = timestamps[indices]
                        n_sequences = max_sequences

                    # Free memory explicitly
                    del feature_data

                    X_list.append(sequences)
                    y_list.append(targets)
                    timestamps_list.append(timestamps)
                    participant_ids_list.append(np.full(len(targets), pid))

                except Exception as e:
                    self.logger.warning(f"Error processing participant {pid}: {str(e)}")
                    continue

            if not X_list:
                raise ValueError("No sequences were created successfully")

            X = np.concatenate(X_list, axis=0)
            y = np.concatenate(y_list, axis=0)
            timestamps = np.concatenate(timestamps_list, axis=0)
            participant_ids = np.concatenate(participant_ids_list, axis=0)

            self.logger.info(f"Created sequences with shape: {X.shape}")
            self.logger.info(f"Target values range: {y.min():.3f} to {y.max():.3f}")

            # GPU Strategy setup
            try:
                self.logger.info("Setting up GPU strategy...")
                policy = tf.keras.mixed_precision.Policy('mixed_float16')
                tf.keras.mixed_precision.set_global_policy(policy)

                physical_devices = tf.config.list_physical_devices('GPU')
                if physical_devices:
                    for device in physical_devices:
                        tf.config.experimental.set_memory_growth(device, True)
                    strategy = tf.distribute.MirroredStrategy()
                    self.logger.info(f"GPU Strategy initialized: {strategy.__class__.__name__}")
                else:
                    strategy = tf.distribute.get_strategy()
                    self.logger.warning("No GPU found, using default strategy")

                n_cores = strategy.num_replicas_in_sync
                self.logger.info(f"Number of devices: {n_cores}")
            except Exception as e:
                self.logger.warning(f"GPU initialization failed: {str(e)}. Falling back to default strategy.")
                strategy = tf.distribute.get_strategy()
                n_cores = 1

            # Memory monitoring callback
            class MemoryMonitorCallback(tf.keras.callbacks.Callback):
                def __init__(self, logger):
                    super().__init__()
                    self.logger = logger

                def on_epoch_end(self, epoch, logs=None):
                    try:
                        import psutil
                        process = psutil.Process()
                        memory_info = process.memory_info()
                        self.logger.info(f"Memory usage after epoch {epoch}: {memory_info.rss / 1024 / 1024:.2f} MB")
                    except:
                        pass

            # Cross-validation
            groups = participant_ids
            gkf = GroupKFold(n_splits=3)
            fold_idx = 0

            for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
                fold_idx += 1
                self.logger.info(f"\n{'='*50}")
                self.logger.info(f"Training fold {fold_idx}/3")
                self.logger.info(f"Training samples: {len(train_idx)}, Validation samples: {len(val_idx)}")

                try:
                    with strategy.scope():
                        batch_size = min(128 * n_cores, 1024)
                        batch_size = (batch_size // n_cores) * n_cores
                        self.logger.info(f"Using batch size: {batch_size}")

                        # Get train/val split
                        X_train, X_val = X[train_idx], X[val_idx]
                        y_train, y_val = y[train_idx], y[val_idx]

                        # Create fold-specific scalers
                        if self.use_robust_scaler:
                            fold_scaler = RobustScaler()
                        else:
                            fold_scaler = StandardScaler()

                        fold_target_scaler = MinMaxScaler()

                        # Scale features for this fold only
                        X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
                        X_train_scaled = fold_scaler.fit_transform(X_train_reshaped)
                        X_train_scaled = np.clip(X_train_scaled, self.clip_range[0], self.clip_range[1])
                        X_train = X_train_scaled.reshape(X_train.shape)

                        # Scale validation with training scaler
                        X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
                        X_val_scaled = fold_scaler.transform(X_val_reshaped)
                        X_val_scaled = np.clip(X_val_scaled, self.clip_range[0], self.clip_range[1])
                        X_val = X_val_scaled.reshape(X_val.shape)

                        # Scale targets
                        y_train = fold_target_scaler.fit_transform(y_train.reshape(-1, 1)).ravel()
                        y_val = fold_target_scaler.transform(y_val.reshape(-1, 1)).ravel()

                        # Create TensorFlow datasets
                        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                        train_dataset = train_dataset.shuffle(buffer_size=10000)
                        train_dataset = train_dataset.batch(batch_size)
                        train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

                        val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
                        val_dataset = val_dataset.batch(batch_size)
                        val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

                        # Save fold-specific scalers if this becomes the best fold
                        if fold_idx == 1 or best_val_metric is None:
                            self.scaler = fold_scaler
                            self.target_scaler = fold_target_scaler

                        self.logger.info("Building model...")
                        model = self.build_model(X_train.shape[1], X_train.shape[2])
                        if model is None:
                            raise ValueError("Failed to build model")

                        # Create callbacks - FIX: moved after dataset creation
                        callbacks = self.create_training_config(train_dataset)
                        callbacks.extend([
                            tf.keras.callbacks.CSVLogger(
                                str(self.run_folder / f'training_log_fold_{fold_idx}.csv')
                            ),
                            MemoryMonitorCallback(self.logger)
                        ])

                        history = model.fit(
                            train_dataset,
                            epochs=100,
                            validation_data=val_dataset,
                            callbacks=callbacks,
                            verbose=1
                        )

                        # Log early stopping information
                        stopped_epoch = 0
                        for callback in callbacks:
                            if isinstance(callback, tf.keras.callbacks.EarlyStopping):
                                stopped_epoch = callback.stopped_epoch
                                break

                        if stopped_epoch > 0:
                            self.logger.info(f"Early stopping triggered at epoch {stopped_epoch}")
                        else:
                            self.logger.info("Training completed without early stopping")
                        histories.append(history.history)

                        val_results = model.evaluate(val_dataset, verbose=1)

                        fold_metric = {
                            'fold': fold_idx,
                            'val_loss': float(val_results[0]),
                            'val_mae': float(val_results[1]),
                            'val_mse': float(val_results[2])
                        }
                        fold_metrics.append(fold_metric)

                        if val_results[0] < best_val_metric:
                            best_val_metric = val_results[0]
                            best_fold = fold_idx

                            # Save best model
                            best_model_dir = self.run_folder / 'best_model'
                            best_model_dir.mkdir(parents=True, exist_ok=True)

                            # Save model architecture
                            model_json = model.to_json()
                            with open(best_model_dir / 'model_architecture.json', 'w') as f:
                                f.write(model_json)

                            # Save weights with correct extension
                            weights_path = str(best_model_dir / 'model.weights.h5')
                            model.save_weights(weights_path)

                            # Save fold indices
                            np.save(str(best_model_dir / 'fold_indices.npy'),
                                {'train': train_idx, 'val': val_idx})

                            best_model = model

                except Exception as e:
                    self.logger.exception(f"Error in fold {fold_idx}")
                    continue
                finally:
                    # Explicit cleanup
                    tf.keras.backend.clear_session()
                    if 'train_dataset' in locals():
                        del train_dataset
                    if 'val_dataset' in locals():
                        del val_dataset
                    import gc
                    gc.collect()

            # Save final metrics and create visualizations
            if fold_metrics:
                metrics_df = pd.DataFrame(fold_metrics)
                metrics_df.to_csv(self.run_folder / 'fold_metrics.csv', index=False)

                # Create performance plots
                self.create_performance_plots(metrics_df, histories)

                self.logger.info("\nTraining Complete!")
                self.logger.info("=" * 50)
                self.logger.info(f"Best fold: {best_fold}")
                self.logger.info(f"Best validation loss: {best_val_metric:.4f}")
                self.logger.info("=" * 50)

            if best_model is None:
                self.logger.error("All training folds failed. No best model available.")
                raise ValueError("Training failed - no best model available")

            return best_model

        except Exception as e:
            self.logger.exception("Error in train_and_evaluate")
            raise  # Re-raise the exception to ensure proper error handling


    def create_performance_plots(self, metrics_df: pd.DataFrame, histories: list):
        """Create and save performance visualization plots."""
        try:
            plots_dir = self.run_folder / 'plots'
            plots_dir.mkdir(exist_ok=True)

            # Plot fold metrics
            metrics = ['val_loss', 'val_mae', 'val_mse']
            for metric in metrics:
                plt.figure(figsize=(10, 6))
                plt.bar(metrics_df['fold'], metrics_df[metric])
                plt.title(f'{metric} by Fold')
                plt.xlabel('Fold')
                plt.ylabel(metric)
                plt.savefig(plots_dir / f'{metric}_by_fold.png')
                plt.close()

            # Plot training history
            for history in histories:
                for metric in history.keys():
                    # Handle 'val_' prefixed metrics
                    if metric.startswith('val_') or metric == 'loss':
                        plt.figure(figsize=(10, 6))
                        # Plot training metric (without 'val_')
                        if metric.startswith('val_'):
                          train_metric = metric.replace('val_', '')
                          if train_metric in history:
                            plt.plot(history[train_metric], label='train')
                            plt.plot(history[metric], label='validation') #plot validation
                          else:
                           continue
                        else:
                            plt.plot(history[metric], label='train')

                            val_metric = 'val_' + metric
                            if val_metric in history:
                                plt.plot(history[val_metric], label='validation')

                        plt.title(f'{metric} During Training')
                        plt.xlabel('Epoch')
                        plt.ylabel(metric)
                        plt.legend()
                        plt.savefig(plots_dir / f'{metric}_history.png')
                        plt.close()

        except Exception as e:
            self.logger.exception("Error creating performance plots")

    def analyze_participant(self, pid: str, model: tf.keras.Model) -> None:
        """Generate prediction plots for a specific participant with proper alignment."""
        try:
            dfp = self.data[self.data['participant_id'] == pid].copy()
            if dfp.empty:
                self.logger.warning(f"No data for participant {pid}")
                return

            # Create sequences with timestamp tracking
            feature_data = dfp[self.features].values
            n_samples = len(dfp)
            if n_samples < self.sequence_length:
                self.logger.warning(f"Insufficient samples for participant {pid}")
                return

            n_sequences = (n_samples - self.sequence_length + self.stride) // self.stride
            sequences = np.zeros((n_sequences, self.sequence_length, len(self.features)))
            target_indices = []  # Track exact indices for alignment

            for i in range(n_sequences):
                start_idx = i * self.stride
                end_idx = start_idx + self.sequence_length
                sequences[i] = feature_data[start_idx:end_idx]
                target_indices.append(end_idx - 1)  # Index where prediction applies

            # Generate predictions
            predictions = model.predict(sequences)
            predictions = self.target_scaler.inverse_transform(predictions)

            # Get actual values at exactly the same indices
            actual_indices = np.array(target_indices)
            actual_values = dfp[self.target].values[actual_indices].reshape(-1, 1)
            actual_values = self.target_scaler.inverse_transform(actual_values)

            # Get timestamps at those exact indices
            timestamps = dfp[self.timestamp_column].values[actual_indices]

            # Plot with perfect alignment
            plt.figure(figsize=(15, 7))
            plt.plot(timestamps, predictions, label='Predicted', alpha=0.7)
            plt.plot(timestamps, actual_values, label='Actual', alpha=0.7)

            # Calculate metrics with perfectly aligned data
            mae = np.mean(np.abs(predictions - actual_values))
            mse = np.mean((predictions - actual_values) ** 2)
            rmse = np.sqrt(mse)

            metrics = {
                'participant_id': pid,
                'mae': float(mae),
                'mse': float(mse),
                'rmse': float(rmse)
            }

            with open(self.run_folder / 'participant_plots' / f'{pid}_metrics.json', 'w') as f:
                json.dump(metrics, f, indent=4)

        except Exception as e:
            self.logger.exception(f"Error analyzing participant {pid}")

def main() -> None:
    """Main execution function with improved stability measures."""
    try:
        # Initialize system with stable configuration
        system = GazeAnalysisSystem(
            window_size=50,
            stride=9,
            model_type="cnn",
            use_robust_scaler=True,
            clip_range=(-3, 3),
            max_seq_per_participant=50000
        )

        # Execute pipeline with proper logging
        logger = logging.getLogger('GazeAnalysisSystem')
        logger.info("Phase 1: Load & preprocess data")
        system.load_and_preprocess_data()

        logger.info("Phase 2: Train & evaluate")
        model = system.train_and_evaluate()

        # Check if model training was successful before proceeding
        if model is None:
            logger.error("Model training failed. Cannot analyze participants.")
            return  # Exit if no model was trained


        logger.info("Phase 3: Evaluate participant generalization")
        generalization_metrics = system.evaluate_participant_generalization()
        logger.info(f"Generalization metrics saved to {system.run_folder / 'participant_generalization.csv'}")

        logger.info("Phase 4: Analyze participants")
        participant_ids = system.data['participant_id'].unique()
        for pid in participant_ids:
            system.analyze_participant(pid, model)

        logger.info(f"Analysis complete. Results saved in {system.run_folder}")

    except Exception as e:
        logger.exception("Error in main execution")
        raise

if __name__ == "__main__":
    main()

In [ ]:
import os
import json
from pathlib import Path
import tensorflow as tf

def find_best_weights(output_dir: str):
    """
    Finds the best weights file in the checkpoints directory.
    """
    output_dir = Path(output_dir)
    checkpoints_dir = output_dir / 'checkpoints'

    if not checkpoints_dir.exists():
        print("No checkpoints directory found")
        return None

    # Get all weight files and parse their numbers correctly
    weight_files = []
    for f in checkpoints_dir.glob('model_*.weights.h5'):
        try:
            # Extract the number between 'model_' and '.weights.h5'
            num = int(f.name.replace('model_', '').replace('.weights.h5', ''))
            weight_files.append((num, f))
        except ValueError:
            print(f"Skipping file with invalid format: {f.name}")

    if not weight_files:
        print("No valid weight files found in checkpoints")
        return None

    # Sort by epoch number and get the latest
    weight_files.sort(key=lambda x: x[0])  # Sort by the extracted number
    best_weights = weight_files[-1][1]  # Take the file path from the tuple with highest number

    print(f"Found best weights file: {best_weights.name}")
    return best_weights

def load_model_safely(output_dir: str, model_type='cnn_transformer'):
    """
    Enhanced model loading with proper weight file handling.
    """
    try:
        print("Initializing model loading process...")
        output_dir = Path(output_dir)

        # Load model architecture
        architecture_path = output_dir / 'best_model' / 'model_architecture.json'
        if architecture_path.exists():
            print("Found model architecture file")
            with open(architecture_path, 'r') as f:
                model_json = f.read()
                model = tf.keras.models.model_from_json(model_json)
                print("Successfully loaded model architecture")
        else:
            print("No architecture file found, rebuilding model...")
            return None

        # Find and load best weights
        best_weights = find_best_weights(output_dir)
        if best_weights:
            try:
                model.load_weights(str(best_weights))
                print(f"Successfully loaded weights from {best_weights.name}")

                # Compile the model after loading weights
                optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=0.5)
                model.compile(
                    optimizer=optimizer,
                    loss='mean_squared_error',
                    metrics=['mean_absolute_error', 'mean_squared_error']
                )
                print("Model compiled successfully")

                return model
            except Exception as e:
                print(f"Error loading weights: {str(e)}")
                return None
        else:
            print("No weights found")
            return None

    except Exception as e:
        print(f"Error in model loading: {str(e)}")
        return None

# Test the loading
run_dir = '/content/drive/MyDrive/output/run_20250225_184902'
model = load_model_safely(run_dir)

if model:
    print("\nModel loaded successfully!")
    print("Model summary:")
    model.summary()
else:
    print("\nFailed to load model properly")

In [ ]:
#!/usr/bin/env python3

import os
import json
import argparse
import numpy as np
import pandas as pd
import joblib
import traceback
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.interpolate import interp1d
from pathlib import Path
from tensorflow.keras.optimizers import Adam

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--drive_path', type=str, default='/content/drive/MyDrive/pilotdata',
                      help="Path to gaze CSV files")
    parser.add_argument('--output_dir', type=str,
                      default='/content/drive/MyDrive/output/run_20250225_184902',  # Change this line
                      help="Path to find the best_model folder")
    parser.add_argument('--model_type', type=str, default='cnn_transformer',
                      choices=['cnn', 'lstm', 'rnn', 'cnn_transformer'],
                      help="Type of model architecture")

    args, _ = parser.parse_known_args()
    return args

def rebuild_model(sequence_length=50, n_features=17, model_type='cnn_transformer'):
    """Rebuilds model using specified architecture."""
    try:
        inputs = tf.keras.layers.Input(shape=(sequence_length, n_features))
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        x = tf.keras.layers.BatchNormalization()(x)

        if model_type == 'cnn':
            # CNN Architecture
            x = tf.keras.layers.Conv1D(32, kernel_size=3, padding="same")(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation("relu")(x)
            x = tf.keras.layers.Dropout(0.2)(x)

            x = tf.keras.layers.Conv1D(64, kernel_size=3, padding="same")(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation("relu")(x)
            x = tf.keras.layers.Dropout(0.2)(x)

            x = tf.keras.layers.GlobalAveragePooling1D()(x)
            x = tf.keras.layers.Dense(32, activation="relu")(x)

        elif model_type == 'lstm':
            # LSTM Architecture
            x = tf.keras.layers.LSTM(32, return_sequences=True)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Dropout(0.2)(x)

            x = tf.keras.layers.LSTM(16)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Dense(16, activation="relu")(x)

        elif model_type == 'rnn':
            # RNN Architecture
            x = tf.keras.layers.SimpleRNN(32, return_sequences=True)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Dropout(0.2)(x)

            x = tf.keras.layers.SimpleRNN(16)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Dense(16, activation="relu")(x)

        else:
            # CNN-Transformer Architecture
            x = tf.keras.layers.Dense(32)(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

            for filters in [32, 64]:
                x = tf.keras.layers.Conv1D(filters=filters, kernel_size=3, padding="same")(x)
                x = tf.keras.layers.BatchNormalization()(x)
                x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
                x = tf.keras.layers.Activation('relu')(x)
                x = tf.keras.layers.Dropout(0.1)(x)

            attention_output = tf.keras.layers.MultiHeadAttention(
                key_dim=32, num_heads=4, dropout=0.1
            )(x, x)
            x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attention_output * 0.5)
            x = tf.keras.layers.GlobalAveragePooling1D()(x)
            x = tf.keras.layers.Dense(32, activation='relu')(x)


        x = tf.keras.layers.BatchNormalization()(x)
        outputs = tf.keras.layers.Dense(1, activation='linear')(x)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        optimizer = Adam(learning_rate=1e-5, clipnorm=0.5)
        model.compile(
            optimizer=optimizer,
            loss='mean_squared_error',
            metrics=['mean_absolute_error', 'mean_squared_error']
        )

        return model

    except Exception as e:
        print(f"Error rebuilding model: {str(e)}")
        traceback.print_exc()
        return None

def find_best_weights(output_dir: str):
    """
    Finds the best weights file in the checkpoints directory.
    """
    output_dir = Path(output_dir)
    checkpoints_dir = output_dir / 'checkpoints'

    if not checkpoints_dir.exists():
        print("No checkpoints directory found")
        return None

    # Get all weight files and parse their numbers correctly
    weight_files = []
    for f in checkpoints_dir.glob('model_*.weights.h5'):
        try:
            # Extract the number between 'model_' and '.weights.h5'
            num = int(f.name.replace('model_', '').replace('.weights.h5', ''))
            weight_files.append((num, f))
        except ValueError:
            print(f"Skipping file with invalid format: {f.name}")

    if not weight_files:
        print("No valid weight files found in checkpoints")
        return None

    # Sort by epoch number and get the latest
    weight_files.sort(key=lambda x: x[0])  # Sort by the extracted number
    best_weights = weight_files[-1][1]  # Take the file path from the tuple with highest number

    print(f"Found best weights file: {best_weights.name}")
    return best_weights

def load_model_safely(output_dir, model_type='cnn_transformer'):
    """Enhanced model loading with architecture support."""
    try:
        print("Initializing model loading process...")
        output_dir = Path(output_dir)

        # Load model architecture
        architecture_path = output_dir / 'best_model' / 'model_architecture.json'
        if architecture_path.exists():
            print("Found model architecture file")
            with open(architecture_path, 'r') as f:
                model_json = f.read()
                model = tf.keras.models.model_from_json(model_json)
                print("Successfully loaded model architecture")
        else:
            print("No architecture file found, rebuilding model...")
            # Fall back to rebuilding model
            model = rebuild_model(
                sequence_length=50,
                n_features=17,
                model_type=model_type
            )
            if model is None:
                raise ValueError("Failed to rebuild model")

        # Find and load best weights
        best_weights = find_best_weights(output_dir)
        if best_weights:
            try:
                model.load_weights(str(best_weights))
                print(f"Successfully loaded weights from {best_weights.name}")

                # Compile the model after loading weights
                optimizer = Adam(learning_rate=1e-5, clipnorm=0.5)
                model.compile(
                    optimizer=optimizer,
                    loss='mean_squared_error',
                    metrics=['mean_absolute_error', 'mean_squared_error']
                )
                print("Model compiled successfully")

                return model
            except Exception as e:
                print(f"Error loading weights: {str(e)}")
                return None
        else:
            print("No weights found")
            return None

    except Exception as e:
        print(f"Critical error in model loading: {str(e)}")
        traceback.print_exc()
        return None

def normalize_scores(scores):
    """Normalize numeric scores to 0-1 range."""
    scores = np.array(scores, dtype=np.float32)
    if scores.max() > 1.0:
        return scores / 100.0
    return scores

def make_predictions_safely(model, sequences, batch_size=128):
    """Memory-efficient batch predictions."""
    try:
        sequences = np.array(sequences, dtype=np.float32)
        if sequences.ndim != 3:
            raise ValueError(f"Expected 3D sequences, got shape {sequences.shape}")

        predictions = []
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i + batch_size]
            batch_predictions = model.predict(batch, verbose=0)
            predictions.append(batch_predictions)

        predictions = np.concatenate(predictions, axis=0).flatten()
        return predictions

    except Exception as e:
        print(f"Error making predictions: {str(e)}")
        traceback.print_exc()
        return None

def create_sequences(df, features, sequence_length):
    """Memory-efficient sequence creation."""
    try:
        data_array = df[features].values.astype(np.float32)
        n_samples = len(data_array)

        if n_samples < sequence_length:
            raise ValueError(f"Not enough samples ({n_samples}) for sequence length {sequence_length}")

        # Use numpy's strided array for memory efficiency
        s = data_array.strides
        sequences = np.lib.stride_tricks.as_strided(
            data_array,
            shape=(n_samples - sequence_length + 1, sequence_length, len(features)),
            strides=(s[0], s[0], s[1]),
            writeable=False
        )

        return sequences.copy()  # Return a contiguous copy

    except Exception as e:
        print(f"Error creating sequences: {str(e)}")
        traceback.print_exc()
        raise

def analyze_eye_movements_with_phases(drive_path, output_dir, model_type='cnn_transformer'):
    """Main analysis function with specified model architecture support."""
    try:
        # Initialize paths
        drive_path = Path(drive_path)
        output_dir = Path('/content/drive/MyDrive/output/run_20250225_184902')
        base_dir = Path('/content/drive/MyDrive/output')  # Base directory for participant data
        movement_dir = output_dir / 'movement_analysis_with_phases'
        movement_dir.mkdir(exist_ok=True, parents=True)

        # Create data directory for CSVs
        data_dir = movement_dir / 'data'
        data_dir.mkdir(exist_ok=True, parents=True)

        # Load model and scaler
        model = load_model_safely(output_dir, model_type)
        if model is None:
            print("Failed to load model. Cannot proceed.")
            return

        try:
            scaler = joblib.load(output_dir / 'feature_scaler.gz')
            print("Loaded feature scaler")
        except Exception as e:
            print(f"No scaler found: {str(e)}")
            scaler = None

        features = [
            'FPOGX', 'FPOGY', 'FPOGD',
            'BPOGX', 'BPOGY',
            'LPCX', 'LPCY', 'LPD', 'LPS',
            'RPCX', 'RPCY', 'RPD', 'RPS',
            'LPMM', 'RPMM',
            'CX', 'CY'
        ]

        # Get participant list
        gaze_files = list(drive_path.glob('*_all_gaze.csv'))
        if not gaze_files:
            print(f"No gaze files found in {drive_path}")
            return

        participant_ids = [f.stem.split('_')[0] for f in gaze_files]
        print(f"Found {len(participant_ids)} participants to analyze")

        # Analysis storage
        all_times = []
        all_predictions = []
        all_actual_scores = []
        all_phases = []
        successful_analyses = []
        failed_analyses = []

        def determine_success(predictions, threshold=0.7, sustained_period=300):
            """
            Determine if the performance is successful based on:
            1. Achieving above threshold performance
            2. Maintaining it for a sustained period
            3. Considering the overall trend
            """
            above_threshold = predictions >= threshold

            # Find the longest sustained period above threshold
            max_sustained = 0
            current_sustained = 0
            success_start_time = None

            for i, is_above in enumerate(above_threshold):
                if is_above:
                    current_sustained += 1
                    if current_sustained > max_sustained:
                        max_sustained = current_sustained
                        if success_start_time is None:
                            success_start_time = i
                else:
                    current_sustained = 0

            # Calculate the percentage of time spent above threshold
            time_above_threshold = np.mean(above_threshold)

            # Consider it successful if:
            # 1. Has a sustained period above threshold
            # 2. Spends at least 30% of total time above threshold
            is_successful = (max_sustained >= sustained_period) and (time_above_threshold >= 0.3)

            return is_successful, (success_start_time if success_start_time else 0) / 60.0  # Convert to minutes

        # Process each participant
        for participant_id in participant_ids:
            try:
                print(f"\nProcessing participant {participant_id}...")

                # Update paths to look in the correct locations
                gaze_file = drive_path / f"{participant_id}_all_gaze.csv"
                scores_file = base_dir / participant_id / f"{participant_id}_detailed_scores.csv"

                if not gaze_file.exists():
                    print(f"Missing gaze file: {gaze_file}")
                    continue
                if not scores_file.exists():
                    print(f"Missing scores file: {scores_file}")
                    continue

                # Load and preprocess data
                df_gaze = pd.read_csv(gaze_file)
                df_scores = pd.read_csv(scores_file)

                # Process timestamps
                time_cols = [col for col in df_gaze.columns if col.startswith('TIME(')]
                if not time_cols:
                    print(f"No TIME column found for {participant_id}")
                    continue

                df_gaze['TIMESTAMP'] = df_gaze[time_cols[0]] - df_gaze[time_cols[0]].iloc[0]
                df_scores['TIMESTAMP'] = df_scores['Time'] - df_scores['Time'].iloc[0]

                # Merge data
                df_merged = pd.merge_asof(
                    df_gaze.sort_values('TIMESTAMP'),
                    df_scores[['TIMESTAMP', 'Phase', 'Row_Total_Score']].sort_values('TIMESTAMP'),
                    on='TIMESTAMP',
                    direction='nearest',
                    tolerance=1.0
                )

                # Clean and prepare
                df_merged = df_merged.dropna(subset=['Row_Total_Score'])
                df_merged[features] = df_merged[features].ffill().bfill().fillna(0)

                # Scale features if scaler available
                if scaler is not None:
                    df_merged[features] = scaler.transform(df_merged[features])

                # Normalize scores
                df_merged['Row_Total_Score'] = normalize_scores(df_merged['Row_Total_Score'])

                # Create sequences
                sequence_length = 50
                X_sequences = create_sequences(df_merged, features, sequence_length)

                # Make predictions
                predictions = make_predictions_safely(model, X_sequences)
                if predictions is None:
                    print(f"Skipping participant {participant_id} due to prediction error")
                    continue

                # Align predictions with data
                df_merged = df_merged.iloc[sequence_length - 1:].copy()
                df_merged['Predicted_Score'] = predictions
                df_merged['Time_Minutes'] = df_merged['TIMESTAMP'] / 60.0

                times = df_merged['Time_Minutes'].values
                phases = df_merged['Phase'].values
                actual_scores = df_merged['Row_Total_Score'].values
                predictions = df_merged['Predicted_Score'].values

                # Save participant data to CSV
                participant_data = pd.DataFrame({
                    'Time_Minutes': times,
                    'Predicted_Score': predictions,
                    'Actual_Score': actual_scores,
                    'Phase': phases
                })

                # Determine success and get the critical time point
                is_successful, success_time = determine_success(predictions)
                success_status = "Successful" if is_successful else "Unsuccessful"

                # Add success information to the dataframe
                participant_data['Success_Status'] = success_status
                participant_data['Success_Time'] = success_time if is_successful else None

                # Save detailed participant data
                csv_filename = f'{participant_id}_movement_analysis_data.csv'
                participant_data.to_csv(data_dir / csv_filename, index=False)
                print(f"Saved data for participant {participant_id} to {csv_filename}")

                # Create plot
                plt.figure(figsize=(15, 6))

                plt.plot(times, predictions, '-', color='blue', alpha=0.7, label='Predicted Score')
                plt.plot(times, actual_scores, '-', color='green', alpha=0.7, label='Actual Score')

                # Add threshold line
                plt.axhline(y=0.7, color='red', linestyle='--', label='Success Threshold (0.7)')

                # If successful, mark the success point
                if is_successful:
                    plt.axvline(x=success_time, color='green', linestyle=':', label=f'Success Point: {success_time:.2f} min')

                plt.ylim(0, 1.0)
                plt.ylabel('Score (0-1)')

                # Mark phases
                phase_changes = df_merged['Phase'].ne(df_merged['Phase'].shift()).cumsum()
                phase_boundaries = df_merged.groupby(phase_changes)['Time_Minutes'].agg(['first', 'last']).reset_index(drop=True)
                for idx, row in phase_boundaries.iterrows():
                    start = row['first']
                    end = row['last']
                    phase_name = df_merged.loc[df_merged['Time_Minutes'] == start, 'Phase'].values[0]
                    plt.axvspan(start, end, color='yellow', alpha=0.1)
                    plt.text((start + end) / 2, 0.98, phase_name,
                            ha='center', va='top', fontsize=9)

                plt.title(f'Eye Movement Timeline - Participant {participant_id} ({success_status})')
                plt.xlabel('Time (minutes)')
                plt.grid(True, alpha=0.3)
                plt.legend(loc='upper right')
                plt.tight_layout()

                # Save plot
                plot_filename = f'{participant_id}_movement_analysis_success.png'
                plt.savefig(movement_dir / plot_filename)
                plt.close()
                print(f"Saved plot for participant {participant_id} - {success_status}")

                # Store data for final summary
                all_times.append(times)
                all_predictions.append(predictions)
                all_actual_scores.append(actual_scores)
                all_phases.append(phases)
                successful_analyses.append(participant_id)

            except Exception as e:
                print(f"Error processing participant {participant_id}: {str(e)}")
                failed_analyses.append(participant_id)
                continue

        # Post-processing across all participants
        if len(all_times) > 0:
            max_time = max([t.max() for t in all_times])
            time_grid = np.linspace(0, max_time, num=1000)

            # Interpolate predictions
            interpolated_predictions = []
            for times, preds in zip(all_times, all_predictions):
                f = interp1d(times, preds, kind='linear', bounds_error=False, fill_value=np.nan)
                interp_pred = f(time_grid)
                interpolated_predictions.append(interp_pred)

            interpolated_predictions = np.array(interpolated_predictions)
            mean_predictions = np.nanmean(interpolated_predictions, axis=0)
            std_predictions = np.nanstd(interpolated_predictions, axis=0)

            # Save interpolated data for all participants
            interpolated_data = pd.DataFrame({
                'Time_Minutes': time_grid,
                'Mean_Predicted_Score': mean_predictions,
                'Standard_Deviation': std_predictions,
                'Number_of_Participants': np.sum(~np.isnan(interpolated_predictions), axis=0)
            })

            # Add individual participant data as additional columns
            for i, participant_id in enumerate(successful_analyses):
                interpolated_data[f'Participant_{participant_id}'] = interpolated_predictions[i]

            # Save detailed interpolated data
            interpolated_data.to_csv(data_dir / 'all_participants_interpolated_data.csv', index=False)
            print(f"Saved interpolated data for all participants")

            # Find peak
            peak_index = np.nanargmax(mean_predictions)
            peak_time = time_grid[peak_index]
            peak_value = mean_predictions[peak_index]
            print(f"\nPeak prediction at {peak_time:.2f} min with value {peak_value:.2f}")

            # Calculate correlation
            all_actual_flat = np.concatenate(all_actual_scores)
            all_preds_flat = np.concatenate(all_predictions)
            correlation = np.corrcoef(all_actual_flat, all_preds_flat)[0, 1]

            # Save summary statistics
            summary_stats = {
                'Total_Participants': len(successful_analyses),
                'Failed_Participants': len(failed_analyses),
                'Peak_Time': peak_time,
                'Peak_Value': peak_value,
                'Overall_Correlation': correlation
            }

            # Add participant-wise success information
            for participant_id in successful_analyses:
                participant_data = pd.read_csv(data_dir / f'{participant_id}_movement_analysis_data.csv')
                success_status = participant_data['Success_Status'].iloc[0]
                success_time = participant_data['Success_Time'].iloc[0]
                summary_stats[f'{participant_id}_Success'] = success_status
                summary_stats[f'{participant_id}_Success_Time'] = success_time

            # Save summary statistics
            pd.DataFrame([summary_stats]).to_csv(data_dir / 'analysis_summary.csv', index=False)
            print("Saved analysis summary")

            # Plot and save average predictions
            plt.figure(figsize=(10, 6))
            plt.plot(time_grid, mean_predictions, color='blue', label='Avg Predicted Score')
            plt.fill_between(time_grid,
                           mean_predictions - std_predictions,
                           mean_predictions + std_predictions,
                           color='blue', alpha=0.2, label='±1 SD')
            plt.axhline(y=0.7, color='red', linestyle='--', label='Success Threshold (0.7)')
            plt.plot(peak_time, peak_value, 'ko', label=f'Peak={peak_time:.2f} min')
            plt.title('Average Predicted Score Over Time')
            plt.xlabel('Time (minutes)')
            plt.ylabel('Score (0-1)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.ylim(0, 1.0)
            plt.tight_layout()

            # Save average predictions plot
            plt.savefig(movement_dir / 'average_prediction_over_time.png')
            plt.close()

            # Save average predictions plot data
            average_plot_data = pd.DataFrame({
                'Time_Minutes': time_grid,
                'Mean_Predicted_Score': mean_predictions,
                'Standard_Deviation': std_predictions,
                'Success_Threshold': np.full_like(time_grid, 0.7),
                'Peak_Time': peak_time,
                'Peak_Value': peak_value
            })
            average_plot_data.to_csv(data_dir / 'average_predictions_plot_data.csv', index=False)
            print("Saved average predictions plot data")

        # Print summary
        print("\nMovement analysis complete!")
        print(f"Results saved in: {movement_dir}")
        print("Analysis Complete:")
        print(f"Successfully analyzed: {len(successful_analyses)} participants")
        print(f"Failed to analyze: {len(failed_analyses)} participants")

        if failed_analyses:
            print("\nFailed participants:")
            for pid in failed_analyses:
                print(f"- {pid}")

        # Save final analysis status to CSV
        final_status = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'total_participants': len(participant_ids),
            'successful_analyses': len(successful_analyses),
            'failed_analyses': len(failed_analyses),
            'failed_participant_ids': ','.join(failed_analyses) if failed_analyses else 'None'
        }
        pd.DataFrame([final_status]).to_csv(data_dir / 'final_analysis_status.csv', index=False)
        print("\nFinal analysis status saved to final_analysis_status.csv")

    except Exception as e:
        print(f"Critical error in analysis: {str(e)}")
        traceback.print_exc()

        # Save error information
        try:
            error_info = {
                'error_message': str(e),
                'traceback': traceback.format_exc(),
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            # Ensure data directory exists even if error occurred early
            data_dir = output_dir / 'movement_analysis_with_phases' / 'data'
            data_dir.mkdir(exist_ok=True, parents=True)

            # Save error log to CSV
            pd.DataFrame([error_info]).to_csv(data_dir / 'error_log.csv', index=False)
            print("Error information saved to error_log.csv")
        except Exception as inner_e:
            print(f"Failed to save error information: {str(inner_e)}")

def main():
    args = parse_args()
    analyze_eye_movements_with_phases(args.drive_path, args.output_dir, args.model_type)

if __name__ == "__main__":
    main()